# Spotify Authentication and Playlist Listing

This notebook helps you:
1. Set up Spotify API credentials
2. Authenticate with Spotify
3. List all your playlists

## Setup Instructions

1. Go to [Spotify Developer Dashboard](https://developer.spotify.com/dashboard)
2. Create a new application
3. Get your Client ID and Client Secret
4. Add `http://127.0.0.1:8888/callback` to your app's Redirect URIs in the settings
5. Create a `.env` file in the same directory as this notebook with:
```
SPOTIFY_CLIENT_ID='your_client_id_here'
SPOTIFY_CLIENT_SECRET='your_client_secret_here'
```

In [2]:
import os
import base64
import json
import requests
from urllib.parse import urlencode
from dotenv import load_dotenv
import spotipy

# Load environment variables
load_dotenv()

# Get Spotify credentials
CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
REDIRECT_URI = 'http://127.0.0.1:8888/callback'

# Set up authentication scope
SCOPE = "playlist-modify-public playlist-modify-private playlist-read-private playlist-read-collaborative"

# Step 1: Create the authorization URL
auth_params = {
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': REDIRECT_URI,
    'scope': SCOPE,
    'show_dialog': 'true'
}

auth_url = 'https://accounts.spotify.com/authorize?' + urlencode(auth_params)

print("Please open this URL in your browser:")
print(auth_url)
print("\nAfter authorizing, you will be redirected to a URL that starts with your redirect URI.")
print("Copy the entire URL from your browser address bar and paste it below.")

# Step 2: Get the authorization code from the redirected URL
redirect_url = input("\nPaste the full URL you were redirected to: ")

# Extract the code from the URL
try:
    code = redirect_url.split('code=')[1].split('&')[0]  # Extract code parameter
    print("Authorization code extracted successfully!")
except IndexError:
    print("Error: Could not extract authorization code from the URL.")
    print("Make sure you copied the full URL after authorizing.")

# Step 3: Request an access token using the authorization code
token_url = 'https://accounts.spotify.com/api/token'
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()

token_data = {
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': REDIRECT_URI
}

headers = {
    'Authorization': f'Basic {auth_header}',
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Make the token request
token_response = requests.post(token_url, data=token_data, headers=headers)

if token_response.status_code == 200:
    token_info = token_response.json()
    access_token = token_info['access_token']
    refresh_token = token_info['refresh_token']
    expires_in = token_info['expires_in']
    
    # Save token info to a cache file
    with open('.spotify_cache', 'w') as f:
        json.dump(token_info, f)
    
    print("Authentication successful! Access token acquired.")
    print(f"Token expires in {expires_in} seconds.")
else:
    print(f"Error: {token_response.status_code} - {token_response.text}")
    raise Exception("Failed to get access token")

# Initialize Spotify client with the access token
sp = spotipy.Spotify(auth=access_token)

Please open this URL in your browser:
https://accounts.spotify.com/authorize?client_id=33cc6ae7668e42f680f90f14eee2c40d&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8888%2Fcallback&scope=playlist-modify-public+playlist-modify-private+playlist-read-private+playlist-read-collaborative&show_dialog=true

After authorizing, you will be redirected to a URL that starts with your redirect URI.
Copy the entire URL from your browser address bar and paste it below.



Paste the full URL you were redirected to:  http://127.0.0.1:8888/callback?code=AQDRJyB_nDDzWiTtfD77zug6zGAnTnhwoC1abikftxOaxtfjAe4w2yoxcTlJPAIV2P1QYi_hAkZfPNJmC_qhiXKSzs_LiVZPiDy75XD_mkbWODeWuCtdFud1EswGYyB_qcIOGz9nKS6-TwvLqSGqLm3l3dCHh_lMuq46KI3MhiV7owh7qnmwe_cRcNPPvw2ViZFReGu32-gxt4NMsY2vXiWqDuWmv-r4tgUYk4JDW2g1uaR11ybo47M4n_qOyWpwc9ONYiOFm9jK3tL5TohzXvTq83ktbZWhvCmVwa0UJolYjt4210-0


Authorization code extracted successfully!
Authentication successful! Access token acquired.
Token expires in 3600 seconds.


In [3]:
# Get user's playlists
def get_all_playlists():
    playlists = []
    results = sp.current_user_playlists()
    
    while results:
        playlists.extend(results['items'])
        if results['next']:
            results = sp.next(results)
        else:
            break
            
    return playlists

# Get and display all playlists
playlists = get_all_playlists()
print("Your Playlists:")
print("-" * 80)
for playlist in playlists:
    print(f"Name: {playlist['name']}")
    print(f"ID: {playlist['id']}")
    print(f"Tracks: {playlist['tracks']['total']}")
    print("-" * 80)

Your Playlists:
--------------------------------------------------------------------------------
Name: Hindi songs (best of 2000-2025)
ID: 0i2S0eEdftTrmLKueMWUKX
Tracks: 363
--------------------------------------------------------------------------------
Name: Travel Songs (Hindi) | Bollywood Roadtrip  Hindi | Indie Travels 
ID: 3IpDoXyKOPgxJvUJYsagyM
Tracks: 116
--------------------------------------------------------------------------------
Name: KK × Atif × Mohit Chauhan🌙
ID: 2GN96kn6Rmt5A1DeBvt2Lg
Tracks: 162
--------------------------------------------------------------------------------
Name: Indie Hindi
ID: 4q64DPIcsampYSLcDT1y3Y
Tracks: 13
--------------------------------------------------------------------------------
Name: Best of MTV Unplugged India
ID: 2FSXLWtee5hkVvg01Dieo8
Tracks: 58
--------------------------------------------------------------------------------
Name: B.a.-ba
ID: 2xTCNY73mqIXA5ghYX17fR
Tracks: 6
-----------------------------------------------------------

## Next Steps

1. Copy the playlist ID you want to sort
2. Open `spotify_playlist_sorter.ipynb`
3. Follow the instructions there to sort your playlist